# E2E Serverless ML pipeline  - Ingest, Train, Auto Deploy Model
  --------------------------------------------------------------------

Using the classic Iris dataset to demonstrate definition and automation of an end to end ML pipeline.

#### **notebook how-to's**
* Write and test ML pipeline in a notebook.
* Use hyper parameter tests
* Convert the code to serverless functions and run in the cluster
* Define an ML pipeline DAG (using KubeFlow Pipelines)
  * with 4 steps: data prep, training, model deployment, model report
* Check our pipeline results from the notebook

<a id='top'></a>
#### **steps**
**[define a new function and its dependencies](#define-function)**<br>
**[run the data collection and training locally](#test-locally)**<br>
**[running a task with Hyper parameters (GridSearch)](#hyper-param)**<br>
**[define cluster jobs, build images and run](#build)**<br>
**[Create a multi-stage KubeFlow Pipeline from our functions](#pipeline)**<br>

In [1]:
# nuclio: ignore
import nuclio

<a id='define-function'></a>
### **define a new function and its dependencies**

In [2]:
%%nuclio cmd 
pip install sklearn
pip install xgboost
pip install matplotlib

In [3]:
%nuclio config spec.build.baseImage = "mlrun/mlrun"

%nuclio: setting spec.build.baseImage to 'mlrun/mlrun'


In [4]:
# use this to supress XGB FutureWarning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [5]:
import xgboost as xgb
import os
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score
from mlrun.artifacts import TableArtifact, PlotArtifact
import pandas as pd
from cloudpickle import dumps


def iris_generator(context):
    iris = load_iris()
    iris_dataset = pd.DataFrame(data=iris.data, columns=iris.feature_names)
    iris_labels = pd.DataFrame(data=iris.target, columns=['label'])
    iris_dataset = pd.concat([iris_dataset, iris_labels], axis=1)
    context.logger.info('saving iris dataframe to {}'.format(context.out_path))
    context.log_artifact(TableArtifact('iris_dataset', df=iris_dataset))
    

def xgb_train(context, 
              dataset='',
              model_name='model.bst',
              max_depth=6,
              num_class=4,
              eta=0.2,
              gamma=0.1,
              steps=20):

    df = pd.read_csv(dataset)
    X = df.drop(['label'], axis=1)
    y = df['label']
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2)
    dtrain = xgb.DMatrix(X_train, label=Y_train)
    dtest = xgb.DMatrix(X_test, label=Y_test)

    # Get params from event
    param = {"max_depth": max_depth,
             "eta": eta, "nthread": 4,
             "num_class": num_class,
             "gamma": gamma,
             "objective": "multi:softmax"}

    # Train model
    xgb_model = xgb.train(param, dtrain, steps)

    preds = xgb_model.predict(dtest)
    best_preds = np.asarray([np.argmax(line) for line in preds])

    # log results and artifacts
    context.log_result('accuracy', float(accuracy_score(Y_test, best_preds)))
    
    context.log_artifact('model', body=dumps(xgb_model), 
                         local_path=model_name, labels={'framework': 'xgboost'})
    
    
import matplotlib
import matplotlib.pyplot as plt
from io import BytesIO

def plot_iter(context, iterations, col='accuracy', num_bins=10):
    df = pd.read_csv(BytesIO(iterations.get()))
    x = df['output.{}'.format(col)]
    fig, ax = plt.subplots(figsize=(6,6))
    n, bins, patches = ax.hist(x, num_bins, density=1)
    ax.set_xlabel('Accuraccy')
    ax.set_ylabel('Count')
    context.log_artifact(PlotArtifact('myfig', body=fig))

The following end-code annotation tells ```nuclio``` to stop parsing the notebook from this cell. _**Please do not remove this cell**_:

In [6]:
# nuclio: end-code
# marks the end of a code section

<a id='test-locally'></a>
### run the data collection and training locally

The functions above can be tested locally. Parameters, inputs, and outputs can be specified in the API or the `Task` object.

We use the ```local``` runtime by default, later on we will use a ```job``` runtime for running containers.

In each run we can specify the function, inputs, parameters/hyper-parameters, etc... For more details, see the [mlrun_basics notebook](mlrun_basics.ipynb).

In [7]:
from mlrun import new_function, code_to_function, NewTask, v3io_cred, new_model_server, mlconf, get_run_db, mount_v3io
# for local DB path use 'User/mlrun' instead 
mlconf.dbpath = 'http://mlrun-api:8080'

#### Generate the iris dataset and store in a CSV

In [8]:
out_path='/User/data/xgb'

In [9]:
gen = new_function().run(name='iris_gen', handler=iris_generator, out_path=out_path) 

[mlrun] 2020-06-09 14:03:15,839 starting run iris_gen uid=16cd05741d784b768149f607da88aa0a  -> http://mlrun-api:8080
[mlrun] 2020-06-09 14:03:15,878 .out_path will soon be deprecated, use .artifact_path
[mlrun] 2020-06-09 14:03:15,878 saving iris dataframe to /User/data/xgb
[mlrun] 2020-06-09 14:03:15,899 log artifact iris_dataset at /User/data/xgb/iris_dataset.csv, size: 2776, db: Y



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...da88aa0a,0,Jun 09 14:03:15,completed,iris_gen,v3io_user=adminkind=handlerowner=adminhost=jupyter-697c84dd-kdb7x,,,,iris_dataset


to track results use .show() or .logs() or in CLI: 
!mlrun get run 16cd05741d784b768149f607da88aa0a --project default , !mlrun logs 16cd05741d784b768149f607da88aa0a --project default
[mlrun] 2020-06-09 14:03:15,943 run executed, status=completed


#### define a training task and run locally

In [10]:
task = NewTask(handler=xgb_train, out_path=out_path, inputs={'dataset': gen.outputs['iris_dataset']})
task.with_params(eta=0.1, max_depth=6, gamma=0.1)

In [11]:
run = new_function().run(task)

[mlrun] 2020-06-09 14:03:15,973 starting run mlrun-ee56d8-xgb_train uid=5da29139907a4242818566cef8d79cf7  -> http://mlrun-api:8080
[mlrun] 2020-06-09 14:03:16,053 log artifact model at /User/data/xgb/model.bst, size: 31779, db: Y



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...f8d79cf7,0,Jun 09 14:03:15,completed,mlrun-ee56d8-xgb_train,v3io_user=adminkind=handlerowner=adminhost=jupyter-697c84dd-kdb7x,dataset,eta=0.1max_depth=6gamma=0.1,accuracy=0.26666666666666666,model


to track results use .show() or .logs() or in CLI: 
!mlrun get run 5da29139907a4242818566cef8d79cf7 --project default , !mlrun logs 5da29139907a4242818566cef8d79cf7 --project default
[mlrun] 2020-06-09 14:03:16,084 run executed, status=completed


<a id="hyper-param" ></a>
### running a task with Hyper parameters (GridSearch)

In many cases we want to run our jobs with multiple parameter combination options, we can simply create a task with hyper params (list of possible values per parameter) and MLRun will run all the combinations.

MLRun will store all the results (see the `iteration_results` artifact), you can specify which result is the best (will be treated as the overall task output) using the selection criteria (`max.accuracy` i.e. the one with maximum value as the `accuracy` result).

In [12]:
# test our function locally with multiple parameters
parameters = {
     "eta":       [0.10, 0.20],
     "max_depth": [3, 6, 10],
     "gamma":     [0.1, 0.3],
     }

hyper_task = NewTask(handler=xgb_train, out_path=out_path, inputs={'dataset': gen.outputs['iris_dataset']})
hyper_task.with_hyper_params(parameters, 'max.accuracy')
run = new_function().run(hyper_task)

[mlrun] 2020-06-09 14:03:16,099 starting run mlrun-0f97b9-xgb_train uid=ccac5b5ce4cc47099610f149d5354187  -> http://mlrun-api:8080
> --------------- Iteration: (1) ---------------
[mlrun] 2020-06-09 14:03:16,159 log artifact model at /User/data/xgb/1/model.bst, size: 32427, db: Y

> --------------- Iteration: (2) ---------------
[mlrun] 2020-06-09 14:03:16,217 log artifact model at /User/data/xgb/2/model.bst, size: 29979, db: Y

> --------------- Iteration: (3) ---------------
[mlrun] 2020-06-09 14:03:16,278 log artifact model at /User/data/xgb/3/model.bst, size: 29259, db: Y

> --------------- Iteration: (4) ---------------
[mlrun] 2020-06-09 14:03:16,346 log artifact model at /User/data/xgb/4/model.bst, size: 32571, db: Y

> --------------- Iteration: (5) ---------------
[mlrun] 2020-06-09 14:03:16,435 log artifact model at /User/data/xgb/5/model.bst, size: 31709, db: Y

> --------------- Iteration: (6) ---------------
[mlrun] 2020-06-09 14:03:16,500 log artifact model at /User/data/

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...d5354187,0,Jun 09 14:03:16,completed,mlrun-0f97b9-xgb_train,v3io_user=adminkind=handlerowner=admin,dataset,,best_iteration=11accuracy=0.4666666666666667,modeliteration_results


to track results use .show() or .logs() or in CLI: 
!mlrun get run ccac5b5ce4cc47099610f149d5354187 --project default , !mlrun logs ccac5b5ce4cc47099610f149d5354187 --project default
[mlrun] 2020-06-09 14:03:17,046 run executed, status=completed


<a id="build"></a>
______________________________________________
### **define cluster jobs and build images**

In order to use our function in a cluster we need to package our code and dependencies.

The ```code_to_function``` call will automatically generate a ```function``` object from the current notebook (or a specified file) with its list of dependencies and runtime configuration.

The `.deploy()` command will build the dependencies and image required for running our function.

We use `.apply(mount_v3io())` to attach a v3io (iguazio data fabric) volume to our function. By default v3io will mount the current user home into the `\User` function path.

Alternatively we can use S3 as a data source or target, for that you need to add AWS credentials to the task and specify paths starting with `s3://` e.g.:

    task.with_secrets('file', 'secrets.txt')
    out_path='s3://my-bucket/data'

In [13]:
# create the function from the notebook code + annotations
xgbfn = code_to_function('xgb', kind="job").apply(mount_v3io())

In [14]:
xgbfn.deploy(skip_deployed=True, with_mlrun=False)

[mlrun] 2020-06-09 14:03:19,863 starting remote build, image: .mlrun/func-default-xgb-latest
INFO[0000] Resolved base name mlrun/mlrun:0.4.8 to mlrun/mlrun:0.4.8 
INFO[0000] Resolved base name mlrun/mlrun:0.4.8 to mlrun/mlrun:0.4.8 
INFO[0000] Retrieving image manifest mlrun/mlrun:0.4.8  
INFO[0000] Retrieving image manifest mlrun/mlrun:0.4.8  
INFO[0000] Built cross stage deps: map[]                
INFO[0000] Retrieving image manifest mlrun/mlrun:0.4.8  
INFO[0000] Retrieving image manifest mlrun/mlrun:0.4.8  
INFO[0001] Unpacking rootfs as cmd RUN pip install sklearn requires it. 
INFO[0026] Taking snapshot of full filesystem...        
INFO[0039] Resolving paths                              
INFO[0044] RUN pip install sklearn                      
INFO[0044] cmd: /bin/sh                                 
INFO[0044] args: [-c pip install sklearn]               
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1315 sha256=e19a38319c77bbe56caa8df1eed97d0eec39

True

**run our task using the cluster job**

In [ ]:
task.with_input('dataset', gen.outputs['iris_dataset'])
nrun = xgbfn.run(task, handler='xgb_train', out_path=out_path, watch=True)

<a id="pipeline"></a>
______________________________________________
## Create a multi-stage KubeFlow Pipeline from our functions
* Load Iris dataset into a CSV
* Train a model using XGBoost with Hyper-parameter
* Deploy the model using Nuclio-serving
* Generate a plot of the training results

In [ ]:
import kfp
from kfp import dsl

**create a model serving function from the [model-serving notebook](https://github.com/mlrun/functions/blob/master/xgb_serving/xgb_serving.ipynb)** 

This function will be used in our workflow

In [ ]:
# define a nuclio-serving functions, generated from a remote notebook file
srvfn = new_model_server('iris-serving', 
                         model_class='XGBoostModel', 
                         filename='https://raw.githubusercontent.com/yjb-ds/functions/development/xgb_serving/xgb_serving.ipynb')

# attach to the fabric (to read the model file)
srvfn.apply(mount_v3io())

**define a 4 step workflow with hyper-params**

In [ ]:
@dsl.pipeline(
    name='My XGBoost training pipeline',
    description='Shows how to use mlrun.'
)
def xgb_pipeline(
   eta = [0.1, 0.2, 0.3], gamma = [0.1, 0.2, 0.3]
):

    ingest = xgbfn.as_step(name='ingest_iris', handler='iris_generator',
                          outputs=['iris_dataset'])

    
    train = xgbfn.as_step(name='xgb_train', handler='xgb_train',
                          hyperparams = {'eta': eta, 'gamma': gamma},
                          selector='max.accuracy',
                          inputs = {'dataset': ingest.outputs['iris_dataset']}, 
                          outputs=['model'])

    
    plot = xgbfn.as_step(name='plot', handler='plot_iter',
                         inputs={'iterations': train.outputs['iteration_results']},
                         outputs=['iris_dataset'])

    # deploy the model serving function with inputs from the training stage
    deploy = srvfn.deploy_step(project = 'iris', models={'iris_v1': train.outputs['model']})

#### Create a KubeFlow client and submit the pipeline with parameters

**define the artifacts output path**
the pipeline outputs will be writtento the artifacts path directory, the path can be a file path (require volume mounts) or an object path (v3io://, s3://, ..).

if we specify `{{workflow.uid}}` in the path it will be replaced with the actual workflow ID, this way every workflow run will store artifacts in a unique location for reproducability.

In [ ]:
from mlrun import run_pipeline
artifact_path = 'v3io:///users/admin/mlrun/kfp/{{workflow.uid}}/'
arguments = {'eta': [0.05, 0.10, 0.40, 0.5], 'gamma': [0.1, 0.3, 0.6]}

In [ ]:
id = run_pipeline(xgb_pipeline, arguments, experiment='xgb', artifact_path=artifact_path)

### check the resilts of our pipeline

In [23]:
# connect to the run db 
db = get_run_db().connect()

In [25]:
# query the DB with filter on workflow ID (only show this workflow) 
db.list_runs('', labels=f'workflow={id}').show()

**[back to top](#top)**